In [16]:
# modules required for handling dataframes
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from itertools import product
import itertools
import argparse
import subprocess
from ete3 import NCBITaxa 
from decimal import Decimal
import seaborn as sns
ncbi = NCBITaxa()

In [3]:
%matplotlib inline

In [4]:
#Function to generate taxonomy columns based on NCBITaxa results for NCBI hit dataframe, 
def search_rank_output_name_append_column(df, staxid_column, rank_search):
    """Input df, staxid_column from same df and rank_search (a desired taxonomic rank 
    from each staxid's lineage), outputs taxonomic name corresponding to rank_search or 'Unclassified' if
    unavailable and appends to df row by row"""
    rank_list = []
    for read_index in range(0, len(staxid_column)):
        taxid = ''
        if ';' in str(staxid_column[read_index]):
            taxid = staxid_column[read_index].split(';')[0]
        else:
            taxid = staxid_column[read_index]
        
        taxid_lineage = ''
        taxid_lineage = ncbi.get_lineage(taxid)
        
        names = ''
        names = ncbi.get_taxid_translator(taxid_lineage)
        
        ranks = ''
        ranks = ncbi.get_rank(taxid_lineage) #Dict
        
        ranks2names = ''
        ranks2names = {ranks[k]:names[k] for k in names.keys() & ranks}
        
        if rank_search in ranks2names.keys():
            rank_list.append(ranks2names[rank_search])#if rank in dict, print name
        else:
            rank_list.append('Unclassified')
    df[rank_search] = rank_list
# NOTE: Appending is always slow, try and find a better way e.g df.apply to a column based on staxids column


#count pivot table of dataframe with taxonomic columns: 
#rows - rank names, columns - barcode, sorted by desired class
def generate_ncbi_taxonomy_pivot_wimp(tax_df, rank, bcs, num):
    """From tax_df, generate a pivot table listing num rank counts, sorted by bcs"""
    pivot_table = tax_df.pivot_table(values='seqlen_wimp', 
                                            index=rank, 
                                            columns='barcode_wimp', 
                                            aggfunc='sum', 
                                            fill_value=0)
    pivot_table.columns.name = None
    pivot_table = pivot_table.sort_values(bcs, axis=0, ascending=False).head(n=num)
    return pivot_table


#count pivot table of dataframe with taxonomic columns: 
#rows - rank names, columns - barcode, sorted by desired class
def generate_ncbi_taxonomy_pivot_blast(tax_df, rank, bcs, num):
    """From tax_df, generate a pivot table listing num rank counts, sorted by bcs"""
    pivot_table = tax_df.pivot_table(values='sequence_length_template_blast', 
                                            index=rank, 
                                            columns='barcode_arrangement_blast', 
                                            aggfunc='sum', 
                                            fill_value=0)
    pivot_table.columns.name = None
    pivot_table = pivot_table.sort_values(bcs, axis=0, ascending=False).head(n=num)
    return pivot_table

In [5]:
# put in all input parameters. Here I am showing the code for one sample as an example.
# to generate the final_df for other samples, simply change the basedir and barcode, as all file names just has this two difference between each two samples.
# please note that there are other places of this script that require understanding and hard coding skills which I also commented below.

basedir = '/home/yiheng/data/20181118_FAH84398' # the directory where all the documents of each sequencing run are stored.
barcode = '07' # the barcode for each sample
basename = os.path.basename(basedir)
# each file (blast_output, wimp_output) has a specific naming system which we could capture their difference and only input variable once
file_name = '%s_albacore231.chopped.barcode%s.fasta.20181217.ntblast_output' % (basename, barcode)
seq_sum_dir = os.path.join(basedir, 'basecalled_data/%s_albacore231' % basename, 'sequencing_summary.txt') # the directory of sequencing summary file for each run
blastouput_dir = os.path.join(basedir,'workspace', 'barcode%s' % barcode, file_name) # the directory for .blast_output file
Q_filter_csv_dir = os.path.join(basedir,'workspace', 'barcode%s' % barcode, 'barcode%s_QC07_basecalling_1d_barcode-v1.csv' % barcode) # wimp_QC.csv file directory
WIMP_csv_dir = os.path.join(basedir,'workspace', 'barcode%s' % barcode, 'barcode%s_QC07_classification_wimp_v2-v1.csv' % barcode) # wimp_taxa.csv file directory

In [6]:
# read all the tables as dataframes
seq_sum_df = pd.read_csv(seq_sum_dir, sep='\t')
blastoupt_df = pd.read_csv(blastouput_dir, header=None, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,20,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# different analysis has different way of storing their output data, check how they store the data
WIMP_df = pd.read_csv(WIMP_csv_dir, sep=',')
Q_filter_df = pd.read_csv(Q_filter_csv_dir, sep=',')

In [9]:
# The seq_sum_df normally big and redundant in information, make it smaller first to speed up the analysis
seq_sum_df = seq_sum_df.loc[:, ['read_id', 'sequence_length_template', 'passes_filtering', 'mean_qscore_template', 'barcode_arrangement']]
seq_sum_df_pass = seq_sum_df[seq_sum_df.passes_filtering == True]

In [13]:
# here are the restructuring of the wimp outputs. althought there are quite few hard coded words but they are all part of the wimp output format.
Q_filter_df = Q_filter_df.drop(columns=['filename', 'runid', 'start_time', 'exit_status'])
Q_filter_df = Q_filter_df[Q_filter_df.mean_qscore >= 7] # quality filtering
WIMP_df = WIMP_df.drop(columns=['filename', 'runid', 'barcode', 'name', 'lineage'])
merged_WIMP_df = pd.merge(Q_filter_df, WIMP_df,how='outer',left_on= 'read_id', right_on='readid')
merged_WIMP_df = merged_WIMP_df.drop(columns='readid')

In [14]:
# since the appending of different taxonomic level was too slow, we separated the human reads and unclassified reads, appended them manually
merged_WIMP_df_no_human = merged_WIMP_df[(merged_WIMP_df.taxID != 9606) & (merged_WIMP_df.exit_status == 'Classified')]
merged_WIMP_df_human = merged_WIMP_df[(merged_WIMP_df.taxID == 9606) & (merged_WIMP_df.exit_status == 'Classified')]
merged_WIMP_df_unclassified = merged_WIMP_df[merged_WIMP_df.exit_status == 'Unclassified']

In [17]:
# append taxa. Some times could get warning or errors when the ete3 was updated as the local taxonomy database
merged_WIMP_df_no_human = merged_WIMP_df_no_human.reset_index(drop=True)
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(merged_WIMP_df_no_human, 
                                          merged_WIMP_df_no_human.taxID, 
                                          rank)

/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662457 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662457 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662457 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662457 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: 

In [18]:
# manually change the others
merged_WIMP_df_human['superkingdom'] = 'Eukaryota'
merged_WIMP_df_human['phylum'] = 'Chordata'
merged_WIMP_df_human['class'] = 'Mammalia'
merged_WIMP_df_human['order'] = 'Primates'
merged_WIMP_df_human['family'] = 'Hominidae'
merged_WIMP_df_human['genus'] = 'Homo'
merged_WIMP_df_human['species'] = 'Homo sapiens'
merged_WIMP_df_unclassified['superkingdom'] = 'Unclassified'
merged_WIMP_df_unclassified['phylum'] = 'Unclassified'
merged_WIMP_df_unclassified['class'] = 'Unclassified'
merged_WIMP_df_unclassified['order'] = 'Unclassified'
merged_WIMP_df_unclassified['family'] = 'Unclassified'
merged_WIMP_df_unclassified['genus'] = 'Unclassified'
merged_WIMP_df_unclassified['species'] = 'Unclassified'

/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [20]:
# concatenate the final dataframe for wimp, to separate with the blast dataframe, we added '_wimp' for the title of each column
final_WIMP_df = pd.concat([merged_WIMP_df_human, merged_WIMP_df_no_human, merged_WIMP_df_unclassified], ignore_index=True)
final_WIMP_df.columns = ['%s_wimp' % x for x in final_WIMP_df.columns]

In [21]:
# refine the final wimp dataframe with the sequencing summary dataframe. here we only use the reads that passed QC for both wimp built-in QC and the albacore QC
final_WIMP_df = final_WIMP_df[final_WIMP_df['read_id_wimp'].isin(seq_sum_df_pass['read_id'])]
final_WIMP_df = final_WIMP_df.reset_index(drop=True)

In [22]:
# now we start to deal with the blast output file.
# blast output file doesn't have basic information (i.e. length, quality etc.) for each read, so we take from the sequencing summary dataframe.
# to normalize with wimp result as mentioned above, we straight filter the seq_sum_df with the wimp final df
seq_sum_df_pass_QC = seq_sum_df[seq_sum_df['read_id'].isin(final_WIMP_df['read_id_wimp'])] # slightly different name with seq_sum_df_pass, not good.
seq_sum_df_pass_QC = seq_sum_df_pass_QC.reset_index(drop=True)

In [34]:
# this are the information I mannually chose to stored by BLAST, some of them are not used so I dropped them
blast_header = ['qseqid', 'sseqid', 'evalue', 'bitscore', 'length', 'pident', 'nident', 
                'sgi', 'sacc', 'staxids', 'sscinames', 'scomnames', 'sskingdoms', 'sstart', 'send']
blastoupt_df.columns = blast_header
blastoupt_df = blastoupt_df.drop(columns=['sgi', 'sacc'])

In [35]:
# First we need to deal with the split reads by Porechop
# my strategy is to keep the '_1' reads and discard the '_2' reads so that the reads that has been split at least have one result
blastoupt_df['qseqid'] = blastoupt_df[blastoupt_df.qseqid.str.contains('_2') == False]
blastoupt_df['qseqid'] = blastoupt_df['qseqid'].apply(lambda x: str(x).split('_')[0])

In [36]:
# also refine the blast output file with the passed QC read list
blastoupt_df = blastoupt_df[blastoupt_df['qseqid'].isin(final_WIMP_df['read_id_wimp'])]

In [37]:
# again, the same strategy to speed up the analysis
blastoupt_df_no_human = blastoupt_df[(blastoupt_df.scomnames != 'human')]
blastoupt_df_human = blastoupt_df[(blastoupt_df.scomnames == 'human')]

In [38]:
blastoupt_df_no_human = blastoupt_df_no_human.drop(columns=['sscinames', 'scomnames', 'sskingdoms'])
blastoupt_df_human = blastoupt_df_human.drop(columns=['sscinames', 'scomnames', 'sskingdoms'])
blastoupt_df_no_human = blastoupt_df_no_human.reset_index(drop=True)
blastoupt_df_human = blastoupt_df_human.reset_index(drop=True)

In [39]:
#ntblasthit_reads_filtered_barcodes_added_TaxaRank = ntblasthit_reads_filtered_barcodes.copy()
# pretty slow
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(blastoupt_df_no_human, 
                                          blastoupt_df_no_human.staxids, 
                                          rank)

/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662456 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662456 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662456 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1662456 was translated into 553814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/yiheng/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: 

In [40]:
blastoupt_df_no_human['barcode'] = 'BC%s' % barcode
blastoupt_df_human['barcode'] = 'BC%s' % barcode
blastoupt_df_no_mammal = blastoupt_df_no_human[blastoupt_df_no_human['class'] != 'Mammalia']
blastoupt_df_no_mammal = blastoupt_df_no_mammal.reset_index(drop=True)
blastoupt_df_mammal = blastoupt_df_no_human[blastoupt_df_no_human['class'] == 'Mammalia']
blastoupt_df_mammal = blastoupt_df_mammal.reset_index(drop=True)

In [42]:
blastoupt_df_human['superkingdom'] = 'Eukaryota'
blastoupt_df_human['phylum'] = 'Chordata'
blastoupt_df_human['class'] = 'Mammalia'
blastoupt_df_human['order'] = 'Primates'
blastoupt_df_human['family'] = 'Hominidae'
blastoupt_df_human['genus'] = 'Homo'
blastoupt_df_human['species'] = 'Homo sapiens'

In [43]:
blast_final_df_no_mammal = pd.merge(seq_sum_df_pass_QC, blastoupt_df_no_mammal, how='outer',left_on= 'read_id', right_on='qseqid')
blast_final_df_mammal = pd.merge(seq_sum_df_pass_QC, blastoupt_df_mammal, how='outer',left_on= 'read_id', right_on='qseqid')
blast_final_df_human = pd.merge(seq_sum_df_pass_QC, blastoupt_df_human, how='outer',left_on= 'read_id', right_on='qseqid')

In [44]:
blast_final_df_no_mammal.dropna(inplace=True)
blast_final_df_mammal.dropna(inplace=True)
blast_final_df_human.dropna(inplace=True)
blast_final_df_no_mammal = blast_final_df_no_mammal.drop(columns=['barcode', 'evalue', 'qseqid'])
blast_final_df_mammal = blast_final_df_mammal.drop(columns=['barcode', 'evalue', 'qseqid'])
blast_final_df_human = blast_final_df_human.drop(columns=['barcode', 'evalue', 'qseqid'])

In [45]:
blast_final_df_classified = pd.concat([blast_final_df_no_mammal, blast_final_df_mammal, blast_final_df_human], ignore_index=True)

In [46]:
blast_final_df_unclassified = seq_sum_df_pass_QC[seq_sum_df_pass_QC['read_id'].isin(blast_final_df_classified['read_id']) == False]
blast_final_df_unclassified['superkingdom'] = 'Unclassified'
blast_final_df_unclassified['phylum'] = 'Unclassified'
blast_final_df_unclassified['class'] = 'Unclassified'
blast_final_df_unclassified['order'] = 'Unclassified'
blast_final_df_unclassified['family'] = 'Unclassified'
blast_final_df_unclassified['genus'] = 'Unclassified'
blast_final_df_unclassified['species'] = 'Unclassified'

/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [47]:
final_blast_df = pd.concat([blast_final_df_classified, blast_final_df_unclassified], ignore_index=True)
final_blast_df.columns = ['%s_blast' % x for x in final_blast_df.columns]

In [48]:
final_blast_df['sequence_length_template_blast'] = final_blast_df['sequence_length_template_blast'].astype(int)
final_blast_df['coverage_blast'] = final_blast_df.nident_blast/final_blast_df.sequence_length_template_blast*100

In [49]:
final_df = pd.merge(final_blast_df, final_WIMP_df, how='outer',left_on= 'read_id_blast', right_on='read_id_wimp')

In [ ]:
final_df.to_csv(os.path.join(basedir, 'analysis', 'final_df.barcode%s.csv' % barcode), sep='\t')